In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import cv2
import csv
import numpy as np

In [3]:
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense, Lambda, Dropout, Activation, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
def split(s, seps):
    res = [s]
    for sep in seps:
        s, res = res, []
        for seq in s:
            res += seq.split(sep)
    return res

In [5]:
def img_preprocess(img):
  img = img[60:135,:,:]
  img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
  img = cv2.GaussianBlur(img,  (3, 3), 0)
  img = cv2.resize(img, (200, 66))

  return img

In [6]:
images = []
measurements = []
lines = []
with open('/home/jpalves/Unity/Data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = split(source_path,['\\','/'])[-1]
        current_path = '/home/jpalves/Unity/Data/IMG/' + filename
        image = cv2.imread(current_path)
        
        images.append(img_preprocess(image))
        
        if i ==0:
            c = 0
        elif i == 1:
            c = 0.2
        else:
            c = -0.2

        measurement = float(line[3])
        measurement = measurement+c
        measurements.append(measurement)

print(np.array(images).shape)
print(np.array(measurements).shape)

#Create Augmented Image and measurement (Steering Angle) Array 
augmented_images, augmented_measurements = [],[]
for image,measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)

print(np.array(augmented_images).shape)
print(np.array(augmented_measurements).shape)

X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

(11790, 66, 200, 3)
(11790,)
(23580, 66, 200, 3)
(23580,)


In [7]:
# checkpoint to save model after each epoch
save_best = ModelCheckpoint('/home/jpalves/Unity/model.h5',
                                    monitor='val_loss',
                                    verbose=1,
                                    save_best_only=True,
                                    mode='min')

# stop training if the validation error stops improving.
early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=.00005,
                                   patience=15,
                                   verbose=1,
                                   mode='auto')

callbacks_list = [save_best]
callbacks_list.append(early_stop)
train_split = 0.8

In [ ]:
model = Sequential()
#Data is Normalized and Mean centered
model.add(Lambda(lambda x:((x/255.0)-0.5),input_shape = (66,200,3)))
#Data is cropped such that a part of the sky that is not required is removed
#model.add(Cropping2D(cropping=((70,25),(0,0))))
#Convolution layer with filter of 5X5 and depth of 6 with a relu activation
model.add(Convolution2D(6,(5,5),activation='relu'))
#Max Pooling along a 2X2 filter
model.add(MaxPooling2D((2,2)))
#Convolution layer with filter of 5X5 and depth of 18 with a relu activation
model.add(Convolution2D(18,(5,5),activation='relu'))
#Max Pooling along a 2X2 filter
model.add(MaxPooling2D((2,2)))
#Convolution layer with filter of 3X3 and depth of 36 with a relu activation
model.add(Convolution2D(36,(3,3),activation='relu'))
#Max Pooling along a 2X2 filter
model.add(MaxPooling2D((2,2)))
#Convolution layer with filter of 3X3 and depth of 4 with a relu activation
model.add(Convolution2D(48,(3,3),activation='relu'))
#Max Pooling along a 2X2 filter
model.add(MaxPooling2D((2,2)))
#Dropout layer to reduce overfitting - ignoring half th generated points
model.add(Dropout(0.1))
# Flatten the network
model.add(Flatten())
#Layer 2 512 Nodes with relu activation
model.add(Dense(512))
model.add(Activation('relu'))
#Layer 3 256 Nodes with relu activation
model.add(Dense(256))
model.add(Activation('relu'))
#Layer 4 128 Nodes with relu activation
model.add(Dense(128))
model.add(Activation('relu'))
#Layer 5 32 Nodes with relu activation
model.add(Dense(32))
model.add(Activation('relu'))
#Output layer with a steering angle prediction
model.add(Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.99, epsilon=1e-07, decay=0),
                  loss='mean_squared_error')

In [ ]:
model = load_model('/home/jpalves/Unity/model.h5')

2022-05-25 17:07:35.091145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 17:07:35.098326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 17:07:35.098958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 17:07:35.099915: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the app

In [9]:
#model.compile(loss ='mse',optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=4759))
#model.fit(X_train, y_train, validation_split = 0.2, batch_size=2, shuffle = True, epochs=50)
print(y_train.shape[0]/2.0)
model.fit(X_train,
            y_train,
            epochs=5,
            batch_size = 2,
            verbose=1,
            shuffle = True,
            validation_split=(1 - train_split),
            callbacks=callbacks_list)

11790.0


2022-06-10 09:38:33.126180: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2022-06-10 09:38:33.794329: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


9432/9432 [==============================] - 18s 2ms/step - loss: 0.0206 - val_loss: 0.0169

Epoch 00001: val_loss improved from inf to 0.01689, saving model to /home/jpalves/Unity/model.h5
Epoch 2/5
9432/9432 [==============================] - 16s 2ms/step - loss: 0.0172 - val_loss: 0.0170

Epoch 00002: val_loss did not improve from 0.01689
Epoch 3/5
9432/9432 [==============================] - 17s 2ms/step - loss: 0.0163 - val_loss: 0.0177

Epoch 00003: val_loss did not improve from 0.01689
Epoch 4/5
9432/9432 [==============================] - 16s 2ms/step - loss: 0.0156 - val_loss: 0.0178

Epoch 00004: val_loss did not improve from 0.01689
Epoch 5/5
9432/9432 [==============================] - 16s 2ms/step - loss: 0.0152 - val_loss: 0.0177

Epoch 00005: val_loss did not improve from 0.01689


In [ ]:
model.save('/home/jpalves/Unity/model.h5')